<a href="https://colab.research.google.com/github/fahmisyuhada/klasifikasi_sel_malaria_CNN/blob/main/klasifikasi_sel_malaria_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!pip install -q kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json
# !kaggle datasets list

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [12]:
!kaggle datasets download -d 'iarunava/cell-images-for-detecting-malaria'
import zipfile

data_set = zipfile.ZipFile('cell-images-for-detecting-malaria.zip', 'r')

data_set.extractall()
data_set.close()
print('dataset unzip successfully')

cell-images-for-detecting-malaria.zip: Skipping, found more recently modified local copy (use --force to force download)
dataset unzip successfully


In [14]:
import numpy as np
np.random.seed

import cv2
import os
from PIL import Image
import keras

os.environ['KERAS_BACKEND'] = 'tensorflow'

image_directory = 'cell_images/'
SIZE = 64
dataset = []
label = []

parasitezed_images = os.listdir(image_directory + 'Parasitized/')
for i, image_name in enumerate(parasitezed_images):
  if(image_name.split('.')[1] == 'png'):
    image = cv2.imread(image_directory + 'Parasitized/' + image_name)
    image = Image.fromarray(image, 'RGB')
    image = image.resize((SIZE, SIZE))
    dataset.append(np.array(image))
    label.append(0)

uninfected_images = os.listdir(image_directory + 'Uninfected/')
for i, image_name in enumerate(uninfected_images):
  if(image_name.split('.')[1] == 'png'):
    image = cv2.imread(image_directory + 'Uninfected/' + image_name)
    image = Image.fromarray(image, 'RGB')
    image = image.resize((SIZE, SIZE))
    dataset.append(np.array(image))
    label.append(1)

print("Load Data Succesfully")

Load Data Succesfully


In [15]:
#####################################
INPUT_SHAPE = (SIZE, SIZE, 3)   # change to (SIZE, SIZE, 3)


###### Make Layer CNN #######
inp = keras.layers.Input(shape=INPUT_SHAPE)

conv1 = keras.layers.Conv2D(32, kernel_size=(3,3), activation='relu', padding='same')(inp)
pool1 = keras.layers.MaxPooling2D(pool_size=(2,2))(conv1)
norm1 = keras.layers.BatchNormalization(axis=-1)(pool1)
drop1 = keras.layers.Dropout(rate=0.2)(norm1)

conv2 = keras.layers.Conv2D(32, kernel_size=(3,3), activation='relu', padding='same')(drop1)
pool2 = keras.layers.MaxPooling2D(pool_size=(2,2))(conv2)
norm2 = keras.layers.BatchNormalization(axis=-1)(pool2)
drop2 = keras.layers.Dropout(rate=0.2)(norm2)

flat = keras.layers.Flatten()(drop2) # Flatten the matrix to get it ready for

hidden1 = keras.layers.Dense(512, activation='relu')(flat)
norm3 = keras.layers.BatchNormalization(axis=-1)(hidden1)
drop3 = keras.layers.Dropout(rate=0.2)(norm3)
hidden2 = keras.layers.Dense(256, activation='relu')(drop3)
norm4 = keras.layers.BatchNormalization(axis=-1)(hidden2)
drop4 = keras.layers.Dropout(rate=0.2)(norm4)

out = keras.layers.Dense(2, activation='sigmoid')(drop4)

model = keras.Model(inputs=inp, outputs=out)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

print(model.summary())



Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 32)        896       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 32, 32, 32)       0         
 2D)                                                             
                                                                 
 batch_normalization_4 (Batc  (None, 32, 32, 32)       128       
 hNormalization)                                                 
                                                                 
 dropout_4 (Dropout)         (None, 32, 32, 32)        0         
                                                                 
 conv2d_3 (Conv2D)           (None, 32, 32, 32)        9248

In [16]:
### Split the dataset
#
# I split the dataset into training and testing dataset.
# 1. Training data: 80%
# 2. Testing data: 20%
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

X_train, X_test, y_train, y_test = train_test_split(dataset, to_categorical(np.array(label)), test_size = 0.20, random_state = 0)

# When training with Keras's Model.fit(), adding the tf.keras.callback.TensorBoard callback
# ensures that logs are created and stored. Additionally, enable histogram computation
#every epoch with histogram_freq=1 (this is off by default)
#Place the logs in a timestamped subdirectory to allow easy selection of different training runs.


In [17]:
#import datetime

#log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "/"
#tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


# ### Training the model
# As the training data is now ready, I will use it to train the model.

#Fit the model
history = model.fit(np.array(X_train),
                         y_train,
                         batch_size = 64,
                         verbose = 1,
                         epochs = 25,      #Changed to 3 from 50 for testing purposes.
                         validation_split = 0.1,
                         shuffle = False
                      #   callbacks=callbacks
                     )

# ## Accuracy calculation
#
# I'll now calculate the accuracy on the test data.

print("Test_Accuracy: {:.2f}%".format(model.evaluate(np.array(X_test), np.array(y_test))[1]*100))

Epoch 1/25
311/311 [==============================] - 181s 573ms/step - loss: 0.5713 - accuracy: 0.7458 - val_loss: 1.0556 - val_accuracy: 0.7800
Epoch 2/25
311/311 [==============================] - 170s 547ms/step - loss: 0.2447 - accuracy: 0.9092 - val_loss: 0.4873 - val_accuracy: 0.8494
Epoch 3/25
311/311 [==============================] - 160s 514ms/step - loss: 0.2000 - accuracy: 0.9259 - val_loss: 0.1631 - val_accuracy: 0.9388
Epoch 4/25
311/311 [==============================] - 182s 587ms/step - loss: 0.1763 - accuracy: 0.9367 - val_loss: 0.2163 - val_accuracy: 0.9270
Epoch 5/25
311/311 [==============================] - 173s 557ms/step - loss: 0.1527 - accuracy: 0.9454 - val_loss: 0.2613 - val_accuracy: 0.9229
Epoch 6/25
311/311 [==============================] - 168s 541ms/step - loss: 0.1406 - accuracy: 0.9496 - val_loss: 0.3270 - val_accuracy: 0.9061
Epoch 7/25
311/311 [==============================] - 167s 539ms/step - loss: 0.1265 - accuracy: 0.9544 - val_loss: 0.1798 -